In [1]:
# Carregando biblioteca e pacotes
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Carregando dataset

retail = pd.read_excel(r"C:\Users\luans\Workspace\projetos_pessoais\Untitled Folder\cohort\Online Retail.xlsx")

# Exibe primeiras linhas do dataset

retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Da primeira à última coluna do dataset temos: cod numérico de identificação da compra, cod numérico de identificação do item, descrição do item, quantidade comprada, data de compra, preço unitário, chave numérica de identificação do consumidor e país da compra.

Para acompanhar o comportamento de determinado cliente ao longo do tempo precisamos que a variavel CustomerId esteja preenchida, por isso vamos excluir linhas que não possuem essa variável "CustomerID" preenchida.

In [ ]:
retail.dropna(subset=['CustomerID'], inplace=True)

Aqui verificamos um ponto preocupante no dataset,perdemos cerca de 25% das linhas no dataset.Como nesse caso não temos muita ação sobre este problema, vamos apenas dar sequência.

Vamos realizar entender um pouco nossos dados antes de gerar as cohorts.

In [ ]:
# Estatistica descritiva

retail.describe()

Aqui a média de itens por compras é em torno de 12, enquanto a mediana indica 5. Isso mostra que temos uma distribuição bastante assimétrica.

Vamos entender a visão do consumidor, como será que é a distribuição de compras feitas pelos clientes? Quantas compras cada cliente fez?

Vamos contar quantos invoices cada cliente tem:

In [ ]:
retail.groupby(['CustomerID'])['InvoiceNo'].nunique().sort_values()

In [ ]:
invoice_per_customer = retail.groupby(['CustomerID'])['InvoiceNo'].nunique()

invoice_per_customer.describe()

Vemos que há muitos clientes com poucas compras e poucos clientes com muitas compras, claramente assimétrica a distribuição de invoices pelos consumidores. Vamos fazer um histograma para verificar esse comportamento.

In [ ]:
_=plt.hist(invoice_per_customer, bins=50)

É bem sutil, mas existem consumidores com 50, 100 e até 200 compras. Mas a maioria está entre 0 e 5 compras.

Com isso podemos esperar pouca retenção.Vamos seguir para a Análise de Cohort. Para isso vamos agrupar pela chave de identificação do cliente, a chave de identificação da compra e a data da compra:


In [ ]:
# Primeira limpeza: Remover duplicidades

retail = retail[['CustomerID', 'InvoiceNo', 'InvoiceDate']].drop_duplicates()

# Vamos organizar por meses

retail['InvoiceMonth'] = retail['InvoiceDate'].dt.to_period('M')

# Vamos pegar o primeiro mes de compra, ou seja, a entrada do cliente

retail['Cohort'] = retail.groupby('CustomerID')['InvoiceDate'].transform('min').dt.to_period('M')
retail.head()

In [ ]:
# Agora vamos agrupar por cohort e por mês de Invoice

retail_cohort = retail.groupby(['Cohort', 'InvoiceMonth']).agg(n_customers=('CustomerID', 'nunique')).reset_index(drop=False)
retail_cohort.head()

Para facilitar a construção da matriz vamos calcular o periodo e criar uma coluna com ele, por exemplo se a cohor se iniciou em dezembro de 2010, o mês de janeiro de 2011 será o número 1, fevereiro será 2 e assim por diante

In [ ]:
# Calcular o periodo em meses
retail_cohort['Period'] = (retail_cohort['InvoiceMonth'].dt.year - retail_cohort['Cohort'].dt.year) * 12 + (retail_cohort['InvoiceMonth'].dt.month - retail_cohort['Cohort'].dt.month)
retail_cohort.head()

In [ ]:
# Criar uma tabela dinamica

cohort_pivot = retail_cohort.pivot_table(index='Cohort', columns='Period', values='n_customers')
cohort_pivot

In [ ]:
# Pegar os valores da primeira coluna

cohort_size = cohort_pivot.iloc[:,0]
cohort_size

In [ ]:
# Realizar a divisão dos números das outras colunas pelo valor da primeira coluna de sua linha.

retention_matrix = cohort_pivot.divide(cohort_size, axis = 0)
retention_matrix

In [ ]:
# Construir mapa de calor
_=plt.subplots(figsize=(12, 8))

_= sns.heatmap(
    data = retention_matrix,
    mask = retention_matrix.isnull(),
    annot = True,
    fmt = '.0%',
    cmap = 'RdYlGn'
)

_ = plt.title('Matriz de Retenção', size=14)

In [ ]:
De acordo com a análise realizada conseguimos chegar as seguintes conclusões:

Queda de Retenção ao Longo do Tempo: É visível que a retenção dos clientes diminui consistentemente à medida que o período avança. Isso é comum em análises de cohort, pois a maioria dos clientes tende a se engajar mais no início e depois perde interesse ou não retorna.

Melhor Cohort (2010-12): A cohort de dezembro de 2010 mostra uma retenção relativamente alta em comparação com as outras, especialmente nos períodos iniciais e novamente no período 11 (com 50%). Isso pode indicar que campanhas ou produtos lançados nessa época foram mais eficazes em manter o interesse dos clientes.

Dificuldades de Retenção em Cohorts Posteriores: A partir de abril de 2011, a retenção parece declinar de forma mais acentuada, especialmente nos períodos intermediários (períodos 4 a 6). A cohort de novembro de 2011, por exemplo, mostra uma queda drástica, com retenção já muito baixa no segundo período (13%).

Possível Efeito Sazonal: Alguns picos de retenção podem indicar efeitos sazonais ou campanhas específicas que foram mais bem-sucedidas em determinados meses, como o aumento de retenção observado no nono período da cohort de dezembro de 2010.

Oportunidade de Intervenção: Intervenções podem ser necessárias para os cohorts de meados de 2011, onde a retenção é baixa em períodos iniciais. Entender por que a retenção caiu tanto nesses períodos pode ser crucial para melhorar futuras campanhas ou estratégias.

Revisão de Estratégias de Engajamento: Para os cohorts onde a retenção caiu drasticamente após o primeiro ou segundo período, pode ser interessante revisar as estratégias de retenção e engajamento implementadas, para identificar o que está causando a perda rápida de clientes.